In [1]:
from datetime import datetime, timedelta
import pandas as pd
import csv
import requests
import os
import re
from itertools import count
import numpy as np

from itertools import accumulate

from spmf import Spmf
import json
import jsonpickle
import heapq

import pprint
from collections import defaultdict, Counter

import jsonpickle
from bisect import bisect

# Event Representations

In [2]:
# A common class for all Events

class Event:
    def __init__(self, eventtype):
        self.type=eventtype
    
    #Return Attribute value given attribute name
    def getAttrVal(self, attrName):
        return self.attributes.get(attrName,None)

    
# A class that represents a point event
class PointEvent(Event):
    def __init__(self, timestamp, attributes):
        super().__init__("point")
        #self.type = "point"
        self.timestamp = timestamp 
        # dictionary: key=attribute value=attribute value
        self.attributes = attributes 
        
    

# class to represent an interval event
class IntervalEvent(Event):
    def __init__(self, t1, t2, attributes):
        super().__init__("interval")
        #self.type = "interval"
        self.time = [t1,t2] 
        # dictionary: key=attribute value=attribute value
        self.attributes = attributes 

In [3]:
class EventStore:
    
    def __init__(self, eventlist=[]):
        self.attrdict={}
        self.reverseatttrdict={}
        self.events=eventlist

    #should be moved to EventStore
    # hold the list of events, also the dictionaries
    
    # Returns a list of event objects
    # src is a url or directory path, if local is false its url else its path
    # header is list of column names if they are not provided in the dataset
    # The foursquare datasets are all using a differnet encoding that pandas cannot auto identify so for those
    # I thought the simplest thing was just to give this function the df and then use that instead of calling my helper
    # for those cases
    #@staticmethod
    def importPointEvents(self, src, timestampColumnIdx, timeFormat, sep='\t', local=False, header=[], df=None):
        events = []
        # if the df is not provided
        if df is None:
            df = get_dataframe(src, local, sep, header)
        cols = df.columns
        # For each event in the csv construct an event object
        for row in df.iterrows():
            data = row[1]
            attribs = {}
            timestamp = datetime.strptime(data[timestampColumnIdx], timeFormat)
            # for all attributes other tahn time, add them to attributes dict
            for i in range(len(data)):
                if i != timestampColumnIdx:
                    attribs[cols[i]] = data[i]
            # use time stamp and attributes map to construct event object
            e = PointEvent(timestamp, attribs)
            events.append(e)
        self.events=events
        #sequence=Sequence(events)
        self.create_attr_dict()
        #return sequence

    # Returns a list of event objects
    # src is a url or directory path, if local is false its url else its path
    # The foursquare datasets are all using a differnet encoding that pandas cannot auto identify so for those
    # I thought the simplest thing was just to give this function the df and then use that instead of calling my helper
    # for those cases
    #@staticmethod
    def importIntervalEvents(self, src, startTimeColumnIdx, endTimeColumnIdx, timeFormat, sep="\t", local=False, header=[], df=None):
        events = []
        # if the df is not provided
        if df is None:
            df = get_dataframe(src, local, sep, header)
        cols = df.columns
        # For each event in the csv construct an event object
        for row in df.iterrows():
            data = row[1]
            attribs = {}
            # create datetime object for the start and end times of the event
            t1 = datetime.strptime(data[startTimeColumnIdx], timeFormat)
            t2 = datetime.strptime(data[endTimeColumnIdx], timeFormat)
            # for all attributes other than times, add them to attributes dict
            for i in range(len(data)):
                if i != startTimeColumnIdx and i != endTimeColumnIdx:
                    attribs[cols[i]] = data[i]
            # use time stamp and attributes map to construct event object
            e = IntervalEvent(t1, t2, attribs)
            events.append(e)
        self.events=events    
        #sequence=Sequence(events)
        self.create_attr_dict()
        #return sequence

    # Import a dataset that has both interval and point events
    # Returns a list of event objects
    # src is a url or directory path, if local is false its url else its path
    # The foursquare datasets are all using a differnet encoding that pandas cannot auto identify so for those
    # I thought the simplest thing was just to give this function the df and then use that instead of calling my helper
    #@staticmethod
    def importMixedEvents(self, src, startTimeColumnIdx, endTimeColumnIdx, timeFormat, sep="\t", local=False, header=[], df=None):
        events = []
        # if the df is not provided
        if df is None:
            df = get_dataframe(src, local, sep, header)
        cols = df.columns
        # For each event in the csv construct an event object
        for row in df.iterrows():
            data = row[1]
            attribs = {}
            # create datetime object for timestamp (if point events) or t1 and t2 (if interval event)
            # If the endTimeColumnIdx value is NaN ie a float instead of a time string then its a point event
            if type(data[endTimeColumnIdx]) is float:
                t = datetime.strptime(data[startTimeColumnIdx], timeFormat)
                event_type = "point"
            # Otherwise its an interval event
            else:
                t1 = datetime.strptime(data[startTimeColumnIdx], timeFormat)
                t2 = datetime.strptime(data[endTimeColumnIdx], timeFormat)
                event_type = "interval"
            # for all attributes other than times, add them to attributes dict
            ignore=[startTimeColumnIdx, endTimeColumnIdx] # list of indices to be ignored
            attribute_columns = [ind for ind in range(len(data)) if ind not in ignore]
            for i in attribute_columns:
                attribs[cols[i]] = data[i]
            # use time stamp (or t1 and t2) and attributes map to construct event object
            if event_type == "point":
                e = PointEvent(t, attribs)
            else:
                e = IntervalEvent(t1, t2, attribs)
            events.append(e)
        self.events=events   
        #sequence=Sequence(events)
        self.create_attr_dict()
        #return sequence

    #should take an eventlist as input
    # Group events by attributeName, and order them by timestamp
    #@staticmethod
    #should return a list of sequences
    def generateSequence(self, attributeName):
        eventList=self.events
        grouped_by = {}
        # Sort the event list
        eventList = sorted(eventList, key=get_time_to_sort_by)
        for event in eventList:
            value = event.attributes[attributeName]
            # If have seen this value before, append it the list of events in grouped_by for value
            if value in grouped_by:
                grouped_by[value].append(event)
            # otherwise store a new list with just that event
            else:
                grouped_by[value] = [event]
        sequences= list(grouped_by.values())
        seqlist=[]
        for seq in sequences:
            seqlist.append(Sequence(seq, self))
        return seqlist
    
    # Split a long sequence into shorter ones by timeUnit. For example, a sequence may span several days and we want to 
    # break it down into daily sequences. The argument timeUnit can be one of the following strings: “hour”, “day”, 
    # “week”, “month”, “quarter”, and “year”.
    # For interval events I used the start time of the event to determine its category when splitting it
    
    #ZINAT- changes
    #SequenceList represents a list of objects of type Sequence. The sequences are further splitted into
    #sequence objects, this way we can use generate sequences and then splitSequences 
    @staticmethod
    def splitSequences(sequenceLists, timeUnit, record=None):
        if not isinstance(sequenceLists, list):
            sequenceLists=[sequenceLists]
        eventstore=sequenceLists[0].eventstore
        results = []
        resultlist=[]
        timeUnit = timeUnit.lower()
        # Check if the time unit is a valid argument
        valid_time_units = ["hour", "day", "week", "month", "quarter", "year"]
        if timeUnit not in valid_time_units:
            raise ValueError("timeUnit must be hour, day, week, month, quarter, or year")
        
        for sequence in sequenceLists:
            # Sort the events by the timestamp or event start time
            sequenceList= sequence.events
            sequenceList = sorted(sequenceList, key=get_time_to_sort_by)

            # Process the event sequence based on the given time unit
            # Generally, create a map for that time unit and then add each event into that map 
            # (key=time such as May 2021 in case of month, value=sequence) and then return the values of the map as a list
            if timeUnit == "hour":
                hours = {}
                for event in sequenceList:
                    time = get_time_to_sort_by(event)
                    key = (time.hour, time.day, time.month, time.year)
                    insert_event_into_dict(key,hours,event)
                    if record is None:
                        event.attributes["record"]=' '.join([str(k) for k in key])
                    else:
                        event.attributes[record]=str(event.attributes[record])+"_"+' '.join([str(k) for k in key])
                results = list(hours.values())

            elif timeUnit == "day":
                days = {}
                for event in sequenceList:
                    time = get_time_to_sort_by(event)
                    key = (time.day, time.month, time.year)
                    insert_event_into_dict(key,days,event)
                    #print(days)
                    if record is None:
                        event.attributes["record"]=datetime(*(key[::-1])).strftime("%Y%m%d")
                    else:
                        event.attributes[record]=str(event.attributes[record])+"_"+datetime(*(key[::-1])).strftime("%Y%m%d")
                results = list(days.values())

            elif timeUnit == "month":
                months = {}
                for event in sequenceList:
                    time = get_time_to_sort_by(event)
                    key = (time.month,time.year)
                    insert_event_into_dict(key,months,event)
                    if record is None:
                        event.attributes["record"]=str(key[0])+str(key[1])
                    else:
                        event.attributes[record]=str(event.attributes[record])+"_"+str(key[0])+str(key[1])
                results = list(months.values())

            elif timeUnit == "week":
                weeks = {}
                for event in sequenceList:
                    time = get_time_to_sort_by(event)
                    year = time.year
                    week_num = time.isocalendar()[1]
                    key = (year,week_num)
                    insert_event_into_dict(key,weeks,event)
                    if record is None:
                        event.attributes["record"]=str(key[0])+"W"+str(key[1])
                    else:
                        event.attributes[record]=str(event.attributes[record])+"_"+str(key[0])+"W"+str(key[1])
                results = list(weeks.values())

            elif timeUnit == "year":
                years = {}
                for event in sequenceList:
                    time = get_time_to_sort_by(event)
                    key = time.year
                    insert_event_into_dict(key,years,event)
                    if record is None:
                        event.attributes["record"]=str(key)
                    else:
                        event.attributes[record]=str(event.attributes[record])+"_"+str(key)
                results = list(years.values())

            elif timeUnit == "quarter":
                quarters = {}
                for event in sequenceList:
                    time = get_time_to_sort_by(event)
                    year = time.year
                    month = time.month
                    # Determine the year, quarter pair/key for quarter dict
                    # January, February, and March (Q1)
                    if month in range(1, 4):
                        key = (year, "Q1")
                    # April, May, and June (Q2)
                    elif month in range(4, 7):
                        key = (year, "Q2")
                    # July, August, and September (Q3)
                    elif month in range(7,10):
                        key = (year, "Q3")
                    # October, November, and December (Q4)
                    elif month in range(10,13):
                        key = (year, "Q4")
                    # Put the event in the dictionary
                    insert_event_into_dict(key,quarters,event)
                    if record is None:
                        event.attributes["record"]=str(key[0])+str(key[1])
                    else:
                        event.attributes[record]=str(event.attributes[record])+"_"+str(key[0])+str(key[1])
                results = list(quarters.values())
            resultlist.extend(results)
        resultlists= [Sequence(x, eventstore) for x in resultlist]

        return resultlists
    
    def getUniqueValues(self, attr):
        l=list(set(event.getAttrVal(attr) for event in self.events))
        return l
    
    #Assuming we are given a list of events and from those events we create 
    #the mapping and reverse mapping dictionary
    def create_attr_dict(self):
        attr_list=self.events[0].attributes.keys()
        print(attr_list)
        
        for attr in attr_list:
            a=48
            unique_list=[]
            unique_list.extend(self.getUniqueValues(attr))
            unique_list=list(set(unique_list))
            #unique_list.clear()
            
            unicode_dict={}
            reverse_dict={}
            for uniques in unique_list:
                unicode_dict[uniques]=chr(a)
                reverse_dict[chr(a)]=uniques
                a=a+1
            self.attrdict[attr]=unicode_dict
            self.reverseatttrdict[attr]=reverse_dict
            #unicode_dict.clear()                    
    
    def get_event_name(self, attr, eventlist):
        #for val in eventlist:
        #    print(self.reverseatttrdict[attr][val])
        return "".join(self.reverseatttrdict[attr][val] for val in eventlist)

# Sequence Representations

In [4]:
class Sequence():
    _ids = count(0)
    

    def __init__(self,  eventlist, eventstore,sid=None):
        # sequence id
        if sid is None:
            self.sid = next(self._ids)
        else:
            self.sid = sid
        
        self.events = eventlist
        self.eventstore=eventstore
        self.volume=1
        self.seqAttributes={}
        self.seqIndices=[]
    def getEventPosition(self, attr, hash_val):
        for count,event in enumerate(self.events):
            #if event.getAttrVal(attr)==hash_val:
            if self.eventstore.attrdict[attr][event.getAttrVal(attr)]==hash_val:
                return count
        return -1
    
    def setVolume(self, intValue):
        self.volume=intValue
        
    def getVolume(self):
        return self.volume
    
    def increaseVolume(self):
        self.volume += 1 
    
    
    def getUniqueValueHashes(self, attr):
        l=list(set(event.getAttrVal(attr) for event in self.events))
        uniquelist=[self.eventstore.attrdict[attr][elem] for elem in l]
        return uniquelist
    
    #Not sure this will always result in same index, will change if 
    #dictionary is updated
    #since python is unordered
    
    def getHashList(self, attr):
        #l=list(list(event.attributes.keys()).index(attr) for event in self.events)
        l=[event.getAttrVal(attr) for event in self.events]
        hashlist=[self.eventstore.attrdict[attr][elem] for elem in l]
        
        return hashlist
    
    def getValueHashes(self, attr):
        l=list(event.getAttrVal(attr) for event in self.events)
        hashlist=[self.eventstore.attrdict[attr][elem] for elem in l]
        
        return hashlist
        
    
    def getEventsHashString(self, attr):
        s=""
        l=list(event.getAttrVal(attr) for event in self.events)
        #for count,event in enumerate(self.events):
        #    s+=str(event.getAttrVal(attr))+" "
        s+="".join(str(self.eventstore.attrdict[attr][elem]) for elem in l)
        #print(s)
        return s
    
    def convertToVMSPReadablenum(self, attr):
        l=list(event.getAttrVal(attr) for event in self.events)
        s=" -1 ".join(str(self.eventstore.attrdict[attr][elem]) for elem in l)
        #s=""
        #for count,event in enumerate(self.events):
        #    s+=str(event.getAttrVal(attr))+" -1 "
        s+=" -2"
        
        return s
    
    def convertToVMSPReadable(self, attr):
        l=list(event.getAttrVal(attr) for event in self.events)
        s=" ".join(self.eventstore.attrdict[attr][elem] for elem in l)
        #s=""
        #for count,event in enumerate(self.events):
        #    s+=str(event.getAttrVal(attr))+" -1 "
        s+="."
        
        return s
    
    def getPathID(self):
        return self.sid
    
    def matchPathAttribute(self, attr, val):
        # should i use eq?!
        if this.seqAttributes.get(attr)==(val):
            return True
        else:
            return False
        
    def setSequenceAttribute(self,attr, value):
        self.seqAttributes[attr]=value
        
         

    # equivalent to method signature public static int getVolume(List<Sequence> seqs)    
    def getSeqVolume(seqlist):
        return sum(seq.getVolume() for seq in seqlist)
    
    
    # Method equivalent to public String getEvtAttrValue(String attr, int hash) in DataManager.java
    def getEvtAttrValue(self, attr, hashval):
        return self.eventstore.reverseatttrdict[attr][hashval]
        
    # Method equivalent to public List<String> getEvtAttrValues(String attr) in DataManager.java    
    def getEvtAttrValues(self, attr):
        l=[event.getAttrVal(attr) for event in self.events]
        return l
        #hashlist=[self.eventstore.attrdict[attr][elem] for elem in l]
        
        #return list(self.eventstore.reverseatttrdict[attr].values())
    
    # Method equivalent to int getEvtAttrValueCount(String attr) in DataManager.java    
    def getEvtAttrValueCount(self, attr):
        return len(self.eventstore.reverseatttrdict[attr])
    
    def getEventsString(self, attr):
        s=""
        l=list(event.getAttrVal(attr) for event in self.events)
        #for count,event in enumerate(self.events):
        #    s+=str(event.getAttrVal(attr))+" "
        s+="-".join(elem for elem in l)
        #print(s)
        return s
    
    
    @staticmethod
    
    def getUniqueEvents(seqlist):
        l=list(set(event.getAttrVal(attr) for event in seq for seq in seqlist))
        return l
    
    

In [5]:
class Pattern:
    _pids = count(1)

    def __init__(self, events=[]):
        #pattern id
        self.id = next(self._pids)
        
        self.keyEvts = events
        
        self.medianPos=[]
        self.meanPos=[]
        
        self.sids=[]
        
        self.support=0
        self.supPercent=None
        self.cluster=None
        self.medianPathLength=0
        self.meanPathLength=0
        
        self.parnetSegment=None
        self.segSizes=None
        
    def filterPaths(self, paths, evtType):
        print("filtering "+ str(len(paths))+" paths by "+str(len(self.keyEvts))+" checkpoints")
        
        for sequences in paths:
            if(self.matchMilestones(sequences.getValueHashes(evtType),self.keyEvts)==False):
                continue
            self.sids.append(sequences)
            
        print(str(len(self.sids))+" matching paths")

        
    def matchMilestones(self, arr, milestones):
        ja=arr
        idx=-1
        for elems in milestones:
            try:
                idx=arr[idx+1:].index(elems)
                #print(idx)
            except ValueError:
                return False
        return True
    
    def getMedianSpacing(self):
        l=[y - x for x,y in zip(self.medianPos,self.medianPos[1:])]
        if(len(l)<=1):
            return 100
        l=l.sort()
        middle=int(len(l)/2)
        if(len(l)%2==0):
            return ((l[middle-1]+l[middle])/2.0)
        else:
            return l[middle]
        return np.median(np.asarray(l))
    
    def addKeyEvent(self, hashval):
        self.keyEvts.append(hashval)
        
    def addToSupportSet(self, seq):
        #print(seq.sid)
        self.sids.append(seq)
        self.support+=seq.getVolume()
        
    def getSequences(self):
        return self.sids
    
    def setMedianPathLength(self, median):
        self.medianPathLength=median
    
    def setMeanPathLength(self, mean):
        self.meanPathLength=mean
        
    def getMedianPathLength(self):
        return self.medianPathLength
    
    def getMeanPathLength(self):
        return self.meanPathLength
    
    def getEvents(self):
        return self.keyEvts
    
    def getEventMeanPos(self):
        return self.meanPos
    
    def getEventMedianPos(self):
        return self.medianPos
    
    #Do we need to preserve order here??

    def getUniqueEventsString(self):
        #return "-".join(str(x) for x in list(set(self.keyEvts)))
        return "-".join(str(x) for x in list(dict.fromkeys(self.keyEvts)))
    
    @staticmethod
    def getPositions(events, path):
        sequence=path
        pos=[]
        idx=-1
        offset=0
        
        for elems in events:
            
            offset+=idx+1
            try:
                idx=path[offset:].index(elems)
            except ValueError:
                continue
            pos.append(offset+idx)
        return pos
    
    def getMedian(self, data):
        #middle=len(data)/2
        #if(len(data)%2==0 and len(data)>1):
        #    return (data[middle-1]+data[middle])/2.0
        #else: 
        #    return data[middle]
        return np.median(data)
    
    def computePatternStats(self, evtAttr):
        pathsOfStrings=[]
        #print(f' sids {self.sids}')
        for path in self.sids:
            pageSequence=path.getHashList(evtAttr)
            pathsOfStrings.append(pageSequence)
        
        #print(f'path of string {pathsOfStrings}')
        medians=[]
        means=[]
        
        ## swap the loops for better readability
        for i,events in enumerate(self.keyEvts):
            numSteps=[]
            
            for idx,paths in enumerate(pathsOfStrings):
                if(self.matchMilestones(paths, self.keyEvts[0:i+1])):
                    pos=self.getPositions(self.keyEvts[0:i+1], paths)
                    if i==0:
                        #add position value of first element id sequence
                        numSteps.append(pos[i])
                    else:
                        #in other cases add the difference
                        numSteps.append(pos[i]-pos[i-1])
            sum_steps=sum(numSteps)
            
            median= self.getMedian(numSteps)
            
            medians.append(median)
            means.append(sum_steps*1.0/ len(numSteps))
                
            
                
        #list(accumulate(means))
        means=np.cumsum(np.asarray(means))
        medians=np.cumsum(np.asarray(medians))
        
        self.setMedianPositions(medians)
        self.setMeanPositions(means)
        
        trailingSteps=[0]*len(self.sids)
        for i,path in enumerate(self.sids):
            pos=self.getPositions(self.keyEvts, path.getHashList(evtAttr))
            #the difference between the last event in thesequence and the last key event
            trailingSteps[i]= len(path.events)- pos[-1]-1
        
        trailStepSum=sum(trailingSteps)
        median= self.getMedian(trailingSteps)
        mean= trailStepSum/len(trailingSteps)
        
        self.setMedianPathLength(median+medians[-1])
        self.setMeanPathLength(mean+means[-1])
                                  
    def getMedianPositions(self, allPos, pids):
        median=[]
        for k in range(0, len(pid)):
            posInPaths=allPos[k]
            median.append(self.getMedian(posInPaths))
        #return list(self.getMedian(posInPaths) for posInPaths in allPos)
        return median
    
    def getMeanPositions(self, allPos, pids):
        mean=[]
        for k in range(0, len(allPos)):
            mean.append(sum(allPos[k])*1.0/(len(allPos[k])))
        return mean
    
    def setMedianPositions(self, median):
        self.medianPos=median
        
    def setMeanPositions(self, mean):
        self.meanPos=mean
        
    def toJson(self):
        return json.dumps(self, default=lambda o: o.__dict__)#,sort_keys=True, indent=4)
    
    def getSupport(self):
        return self.support
    
    def setCluster(self, cluster):
        self.cluster=cluster
        
    def setParent(self, parent, segment):
        self.parent=parent
        self.parentSegment=segment
    
    
    # How to implement this with BitArray?
    #def getEventBitSet(self)
    
    def getParent(self):
        return self.parent
    
    def getParentSegment(self):
        return self.parentSegment
    
    def setMeanPathLength(self,d):
        self.meanPathLength=d
    
    def getMeanPathLength(self):
        return self.meanPathLength
        
    def setSupport(self, sup, total):
        self.support=sup
        self.supPercent= sup*1.0/total
        
    def getEvtAttrValue(self, attr, eventstore):
        return "".join(eventstore.reverseatttrdict[attr][hashval] for hashval in self.keyEvts)    

# Sequence Synopsis

In [6]:
class cluster:
    
    def __init__(self, pat, seq):
        self.pattern = pat
        self.seqList = seq
    
def print_clust(clust_d, attr):
    print("------key val pairs----")
    for key, value in clust_d.items():
        print(f'Pattern {key.keyEvts}')
        for v in value:
            print(f'Sequences {v.getHashList(attr)}')

class SequenceSynopsis:
    
    def minDL(self, Sequences, attr):
        # Initialization Phase
        clust_dict={}
        clust_dict={Pattern(seq.getHashList(attr)): [seq] for seq in Sequences}
        print("Initial clusts")
        print_clust(clust_dict, attr)
        #print(clust_dict.values())

        PriorituQueue=[]

        for c_i in clust_dict.items():
            for c_j in clust_dict.items():
                #print(f'c_i: {c_i}')
                if (c_i[0].id== c_j[0].id):
                    continue
                del_L, c_star=self.merge(c_i,c_j, attr)
                if del_L>0:
                    PriorituQueue.append(tuple((del_L,c_i,c_j,c_star)))
        #print(f'Priority {PriorituQueue}')

        while PriorituQueue:
            PriorituQueue=sorted(PriorituQueue, key= lambda x: x[0], reverse=True)# sort on del_L
            #print(f'Priority {PriorituQueue}')
            del_L,c_i,c_j,c_star= PriorituQueue[0]
            c_new=c_star
            #print(f'PQ values {del_L}, {c_i}, {c_j}, {c_star}')
            #print(f'c_i: {c_i[0]}')
            #clust_dict.pop(c_i[0], None)
            #clust_dict.pop(c_j[0], None)
            print_clust(clust_dict, attr)
            del clust_dict[c_i[0]]
            del clust_dict[c_j[0]]

            clust_dict[c_new[0]]=c_new[1]
            print(f'c_i: {c_i}')
            print(f'c_j: {c_j}')
            delete_indices=[]

            #print(f'PQ before deletion {PriorituQueue}')
            for val, entries in enumerate(PriorituQueue):
                #print(f'val {val} entries {entries}')
                #print(f'entries[1][0] {entries[1][0]} entries[2][0] {entries[2][0]}')
                if c_i[0]==entries[1][0] or c_j[0]==entries[1][0]:
                    #print(f'metched c_i {c_i}')
                    delete_indices.append(val)
                    #del entries
                elif c_i[0]==entries[2][0] or c_j[0]==entries[2][0]:
                    #print(f'metched c_i {c_j}')
                    delete_indices.append(val)
                    #del entries
            #print(f'PQ {PriorituQueue}')

            for index in sorted(delete_indices, reverse=True):
                del PriorituQueue[index]
            #print(f'PQ after deletion {PriorituQueue}')
            #del PriorituQueue[delete_indices]    
            for c in clust_dict.items():
                if c==c_new:
                    continue

                del_L, c_star= self.merge(c, c_new, attr)

                if del_L>0:
                    PriorituQueue.append(tuple((del_L,c,c_new, c_star)))
                    #clust_dict[]
        return clust_dict
    
    def merge(self, pair1, pair2,attr, alpha=1, lambda_val=0):
        key1, val1=pair1
        key2, val2=pair2
        #print(f'key {key1.keyEvts}')
        P_star= Pattern(lcs(key1.keyEvts,key2.keyEvts))
        print(f'Pattern 1 {key1.keyEvts}')
        print(f'Pattern 2 {key2.keyEvts}')
        print(f'Pattern LCS {P_star.keyEvts}')
        E_c=list(((Counter(key1.keyEvts)-Counter(P_star.keyEvts))|(Counter(key2.keyEvts)-Counter(P_star.keyEvts))).elements())
        #print(f'cpunter 1{(Counter(key1.keyEvts)-Counter(P_star.keyEvts))}')
        #print(f'cpunter 2{(Counter(key2.keyEvts)-Counter(P_star.keyEvts))}')
        #print(f'cpunter together{(Counter(key1.keyEvts)-Counter(P_star.keyEvts))|(Counter(key2.keyEvts)-Counter(P_star.keyEvts))}')
        #E_c= ((Counter(key1.keyEvts)-Counter(P_star.keyEvts))|(Counter(key2.keyEvts)-Counter(P_star.keyEvts))).elements()
        #print(f'Counter {E_c}')
        #sort Ec by frequency in desc order;
        #E_c_counter = sort_by_frequency(val1, val2, E_c)
        E_c_counter = Counter(E_c)
        E_c_counter=sorted(E_c_counter, key=E_c_counter.get, reverse=True)
        #print(f'Counter {E_c_counter}')
        del_L=-1

        #P_star= Pattern(P_star)
        lcs_pos_p_i= Pattern.getPositions(P_star.keyEvts,key1.keyEvts )
        lcs_pos_p_j= Pattern.getPositions(P_star.keyEvts,key2.keyEvts )

        print(f'positions p_i {lcs_pos_p_i}')
        print(f'positions p_j {lcs_pos_p_j}')

        for e in E_c_counter:
            #temp_P=P_star

            print(f'candidate {e}')

            indices_key1 = [i for i, x in enumerate(key1.keyEvts) if x == e]
            indices_key2 = [i for i, x in enumerate(key2.keyEvts) if x == e]

            #print(f'indices {indices_key1}')
            #print(f'indices 2{indices_key2}')

            #indices_key1.extend(indices_key2)
            #print(f'all indices {indices_key1}')
            candidate_pos=[]
            for ind in indices_key1:
                if e in P_star.keyEvts and ind in lcs_pos_p_i:
                    continue

                temp_P=P_star
                index=bisect(lcs_pos_p_i, ind)
                #print(f'ind {ind} bisect {index}')
                candidate_pos.append(index)

            for ind in indices_key2:
                if e in P_star.keyEvts and ind in lcs_pos_p_j:
                    continue

                temp_P=P_star
                index=bisect(lcs_pos_p_j, ind)
                #print(f'ind {ind} bisect {index}')
                candidate_pos.append(index)

            candidate_pos= list(set(candidate_pos))

            print(f'candidates {candidate_pos}')

            for index in candidate_pos:
                temp_P.keyEvts.insert(index,e)

                #temp_P.addKeyEvent(e)
                #print(f'temporary pattern {temp_P}')
                del_L_prime= len(key1.keyEvts)+ len(key2.keyEvts)- len(temp_P.keyEvts)+lambda_val
                del_L_prime+= sum(alpha*calc_dist(v1.getHashList(attr), key1.keyEvts) for v1 in val1)
                del_L_prime+= sum(alpha*calc_dist(v2.getHashList(attr), key2.keyEvts) for v2 in val2) #alpha*calc_dist(val2, key2.keyEvts)
                del_L_prime-= sum(alpha*calc_dist(v.getHashList(attr),temp_P.keyEvts) for v in val1+val2)

                print(f'del L prime {del_L_prime}')
                if(del_L_prime<0 or del_L_prime< del_L):
                    break

                else:
                    del_L= del_L_prime
                    P_star=temp_P
                print(f'del L  {del_L}')
                print(f'P_star {P_star.keyEvts}')
                del temp_P.keyEvts[index]


            #return del_L, (P_star, list(set(val1) & set(val2)))
            return del_L, (P_star, val1+val2)



def lcs(a, b):
    tbl = [[0 for _ in range(len(b) + 1)] for _ in range(len(a) + 1)]
    for i, x in enumerate(a):
        for j, y in enumerate(b):
            tbl[i + 1][j + 1] = tbl[i][j] + 1 if x == y else max(
                tbl[i + 1][j], tbl[i][j + 1])
    res = []
    i, j = len(a), len(b)
    while i and j:
        if tbl[i][j] == tbl[i - 1][j]:
            i -= 1
        elif tbl[i][j] == tbl[i][j - 1]:
            j -= 1
        else:
            res.append(a[i - 1])
            i -= 1
            j -= 1
    return res[::-1]

def sort_by_frequency(v1, v2, selectedevents):
    counter=Counter()
    for item in (v1,v2):
        for seqs in item:
            for event in seqs.events:
                if event in selectedevents:
                    counter[event]+=1
    print(counter)
    return sorted(counter, key=counter.get, reverse=True)
    
def levenshtein(s1, s2):
    #From Wikipedia
    if len(s1) < len(s2):
        return levenshtein(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    #print(previous_row)
    return previous_row[-1]    


def calc_dist(seqs, pattern):
    #total=0
    #print(f'val seq {seqs}')
    #print(f'val pattern {pattern}')
    #total= sum(levenshtein(val.events, pattern) for val in seqs)
    total= levenshtein(seqs, pattern)
    #print(f'total {total}')
    return total



# Helper Functions

In [7]:
# Helper function to return a data frame
# Local is boolean, if local then source should be path to the file
# Otherwise it should be a URL to the the file
def get_dataframe( src, local=False, sep="\t", header=[]):
    if not local:
        # To force a dropbox link to download change the dl=0 to 1
        if "dropbox" in src:
            src = src.replace('dl=0', 'dl=1')
        # Download the CSV at url
        req = requests.get(src)
        url_content = req.content
        csv_file = open('data.txt', 'wb') 
        csv_file.write(url_content)
        csv_file.close()
        # Read the CSV into pandas
        # If header list is empty, the dataset provides header so ignore param
        if not header:
            df = pd.read_csv("data.txt", sep)
        #else use header param for column names
        else:
            df = pd.read_csv("data.txt", sep, names=header)
        # Delete the csv file
        os.remove("data.txt")
        return df
    # Dataset is local
    else:
        # If header list is empty, the dataset provides header so ignore param
        if not header:
            print(src)
            df = pd.read_csv(src, sep)
        # else use header param for column names
        else:
            df = pd.read_csv(src, sep, names=header)
        return df
    
    
# Helper function for generateSequence to use when sorting events to get what time field to sort by
# Also used in splitSequences to give the time of an event when splitting the events up

def get_time_to_sort_by(e):
    # Sort by starting time of event if its an interval event
    if type(e) == IntervalEvent:
        return e.time[0]
    # Otherwise use the timestamp
    else:
        return e.timestamp


    
# Helper to insert an event into a map
# Params are key=unique id for that time, map of key to event list, event object
def insert_event_into_dict(key, dictionary, event):
    if key in dictionary:
        dictionary[key].append(event)
    else:
        dictionary[key] = [event]



# Event Aggregation
For aggregateEventsRegex and aggregateEventsDict, see what the files are expected to look like in the repo in DataModel/testFiles

In [8]:
# Helper function to run the mappings file as a dictionary
def give_dictionary_of_mappings_file(fileName):
    # Open the file and split the contents on new lines
    file = open(fileName, "r")
    mappings = file.read().split("\n")
    file.close()
    # Remove any empty strings from the list of mappings
    mappings = list(filter(None, mappings))
    # Raise an error if there is an odd number of items in mapping
    if (len(mappings) % 2) != 0:
        raise ValueError("There must be an even number of lines in the mappings file.")
    # Create a dictionary based on read in mappings
    aggregations = {}
    for i in range(len(mappings)):
        if i % 2 == 0:
            aggregations[mappings[i]] = mappings[i+1]
    #print(aggregations)
    return aggregations

# NOTE: this current modifies the events in eventList argument
# merge events by rules expressed in regular expressions. For example, in the highway incident dataset, we can 
# replace all events with the pattern “CHART Unit [number] departed” by “CHART Unit departed”. The argument 
# regexMapping can be a path pointing to a file defining such rules. We can assume each rule occupies two lines: 
# first line is the regular expression, second line is the merged event name 
def aggregateEventsRegex(eventList, regexMapping, attributeName): 
    aggregations = give_dictionary_of_mappings_file(regexMapping)
    for event in eventList:
        # Get the attribute value of interest
        attribute_val = event.attributes[attributeName]
        # For all the regexes
        for regex in aggregations.keys():
            # If its a match then replace the attribute value for event with
            if re.match(regex, attribute_val):
                event.attributes[attributeName] = aggregations[regex]
                break
    return eventList
    
# NOTE: this current modifies the events in eventList argument
# merge events by a dictionary mapping an event name to the merged name. The argument nameDict can be a path 
# pointing to a file defining such a dictionary. We can assume each mapping occupies two lines: first line is the 
# original name, second line is the merged event name.    
def aggregateEventsDict(eventList, nameDict, attributeName):
    aggregations = give_dictionary_of_mappings_file(nameDict)
    # Iterate over all events and replace evevnts in event list with updated attribute name
    # if directed to by given mappings
    for event in eventList:
        # Get the attribute value of interest
        attribute_val = event.attributes[attributeName]
        # If the attribute value has a mapping then replace the event's current value with the one in give map
        if attribute_val in aggregations:
            
            event.attributes[attributeName] = aggregations[attribute_val]
    return eventList

# Importing events functions

# Generating Sequences

In [9]:
sequence_braiding_Es= EventStore()
sequence_braiding_Es.importPointEvents('../corelow_paper_test.csv', 1, "%m/%d/%y", sep=',', local=True)
#print(type(sequence_braiding))
seq=Sequence(sequence_braiding_Es.events, sequence_braiding_Es)
seq_list=sequence_braiding_Es.generateSequence("Category")

../corelow_paper_test.csv
dict_keys(['Event', 'Category'])


In [10]:
print(sequence_braiding_Es.reverseatttrdict['Event'])

{'0': 'K', '1': 'E', '2': 'H', '3': 'L', '4': 'D', '5': 'C', '6': 'J', '7': 'B', '8': 'A', '9': 'G', ':': 'F', ';': 'I'}


In [11]:
syn=SequenceSynopsis()
G=syn.minDL(seq_list, "Event")

Initial clusts
------key val pairs----
Pattern ['4', '1', '7', '7', '8']
Sequences ['4', '1', '7', '7', '8']
Pattern ['5', '6', '4', ':', '7', '9', '7']
Sequences ['5', '6', '4', ':', '7', '9', '7']
Pattern ['8', '7', '5', '7', '5', '4', '1']
Sequences ['8', '7', '5', '7', '5', '4', '1']
Pattern ['5', '8', '5', '5', '8']
Sequences ['5', '8', '5', '5', '8']
Pattern ['5', '3', ':', '9', '2']
Sequences ['5', '3', ':', '9', '2']
Pattern [';', '0', '8', '1']
Sequences [';', '0', '8', '1']
Pattern 1 ['4', '1', '7', '7', '8']
Pattern 2 ['5', '6', '4', ':', '7', '9', '7']
Pattern LCS ['4', '7', '7']
positions p_i [0, 2, 3]
positions p_j [2, 4, 6]
candidate 1
candidates [1]
del L prime 3
del L  3
P_star ['4', '1', '7', '7']
Pattern 1 ['4', '1', '7', '7', '8']
Pattern 2 ['8', '7', '5', '7', '5', '4', '1']
Pattern LCS ['4', '1']
positions p_i [0, 1]
positions p_j [5, 6]
candidate 7
candidates [0, 2]
del L prime 1
del L  1
P_star ['7', '4', '1']
del L prime 1
del L  1
P_star ['4', '1', '7']
Patter

In [12]:
for key, value in G.items():
    print(f'keys{key.keyEvts}')
    print(key.getUniqueEventsString())
    #print(key.getEvtAttrValue(sequence_braiding_Es,"Event"))
    print(sequence_braiding_Es.get_event_name("Event",key.keyEvts))
    for v in value:
        #print(v.getHashList("Event"))
        print(v.getEvtAttrValues("Event"))

#for key, value in G.items():
#    print(key.keyEvts)
        

keys['5', '3', ':', '9', '2']
5-3-:-9-2
CLFGH
['C', 'L', 'F', 'G', 'H']
keys[';', '0', '8', '1']
;-0-8-1
IKAE
['I', 'K', 'A', 'E']
keys['4', '7', '7']
4-7
DBB
['D', 'E', 'B', 'B', 'A']
['C', 'J', 'D', 'F', 'B', 'G', 'B']
keys['8', '5', '5']
8-5
ACC
['A', 'B', 'C', 'B', 'C', 'D', 'E']
['C', 'A', 'C', 'C', 'A']


In [13]:
def json_default_dump(obj)-> dict:
        return {
            "Pattern": [x.keyEvts for x in obj.keys()],
            "links":obj.values()

        }

def json_serialize_dump(obj):

    if hasattr(obj, "json_default_dump"):

        return obj.json_default_dump()
    if isinstance(obj, set):
        return list(obj)
    return None #obj.__dict_

In [16]:
def key_to_json(data):
    print(type(data))
    if data is None or isinstance(data, (bool, int, str)):
        return data
    if isinstance(data, Pattern):
        print("X")
        return "-".join(str(x) for x in data.keyEvts)
    raise TypeError

def to_json(data):
    print(type(data))
    print(data)
    if data is None or isinstance(data, (bool, int, tuple, range, str)):
        return data
    if isinstance(data, (set, frozenset)):
        return sorted(data)
    if isinstance(data, Sequence):
        print("here")
        return data.getEventsHashString("Event")
    if isinstance(data, list):
        return [to_json(x) for x in data]
    if isinstance(data, dict):
        print("Here")
        return {key_to_json(key): to_json(data[key]) for key in data}
    raise TypeError

data = G
json.dumps(to_json(data))
with open('outfile_sequence_synopsis.json', 'w') as the_file2:
        the_file2.write(json.dumps(to_json(data), indent=1))

<class 'dict'>
{<__main__.Pattern object at 0x7ff1e38e3250>: [<__main__.Sequence object at 0x7ff1e388b090>], <__main__.Pattern object at 0x7ff1e38e3050>: [<__main__.Sequence object at 0x7ff1e388b790>], <__main__.Pattern object at 0x7ff1e3921490>: [<__main__.Sequence object at 0x7ff1e388b050>, <__main__.Sequence object at 0x7ff1e388b590>], <__main__.Pattern object at 0x7ff1e3921ad0>: [<__main__.Sequence object at 0x7ff1e388bc90>, <__main__.Sequence object at 0x7ff1e388bfd0>]}
Here
<class 'list'>
<class '__main__.Sequence'>
here
<class '__main__.Pattern'>
X
<class 'list'>
<class '__main__.Sequence'>
here
<class '__main__.Pattern'>
X
<class 'list'>
[<__main__.Sequence object at 0x7ff1e388b050>, <__main__.Sequence object at 0x7ff1e388b590>]
<class '__main__.Sequence'>
here
<class '__main__.Sequence'>
here
<class '__main__.Pattern'>
X
<class 'list'>
[<__main__.Sequence object at 0x7ff1e388bc90>, <__main__.Sequence object at 0x7ff1e388bfd0>]
<class '__main__.Sequence'>
here
<class '__main__.

In [15]:
x=json.dumps(G, ensure_ascii=False, default= json_serialize_dump, indent=1)
print(x)

TypeError: keys must be str, int, float, bool or None, not Pattern

In [ ]:
f=lcs([1,2,4,5],[1,4,6,7])
print(f)

In [ ]:
f=levenshtein([1,2,4,5],[1,4,6,7])
print(f)

In [ ]:
sequence_braiding_Es= EventStore()
sequence_braiding_Es.importPointEvents('../datasets/sequence_braiding_refined.csv', 0, "%m/%d/%y", sep=',', local=True)
#print(type(sequence_braiding))
seq=Sequence(sequence_braiding_Es.events, sequence_braiding_Es)
seq_list=sequence_braiding_Es.splitSequences(seq, "week")

In [ ]:
G=minDL(seq_list, 'Meal')

In [ ]:
for key, value in G.items():
    print(f'keys{key.keyEvts}')
    for v in value:
        #print(v.getHashList("Meal"))
        print(v.getEvtAttrValues("Meal"))

#for key, value in G.items():
#    print(key.keyEvts)
        

In [ ]:
a = {'ac': 1, 'dc': 2}

In [ ]:
for x in a:
    print(x)